<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/dune/Cypher_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install neo4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# Define Neo4j connections
import pandas as pd
from neo4j import GraphDatabase

host = 'neo4j+s://f4461036.databases.neo4j.io'
user = 'neo4j'
password = 'password'
driver = GraphDatabase.driver(host,auth=(user, password))

def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [5]:
run_query("""
CREATE CONSTRAINT IF NOT EXISTS FOR (c:Character) REQUIRE c.name IS UNIQUE;
""")

""


In [41]:
run_query("""
LOAD CSV WITH HEADERS FROM "https://bit.ly/3kbU21V" AS row
CALL {
  WITH row
  MERGE (c:Character {name:row.Character})
  SET c += apoc.map.clean(row, ["name", "Detail", "to", "relationship_type", "House_Allegiance"], [])
  WITH c, row
  CALL apoc.create.addLabels(c, [row.House_Allegiance]) YIELD node
  CALL {
    WITH row, c
    WITH row, c
    WHERE row.to IS NOT NULL
    MERGE (c1:Character {name: row.to})
    WITH row, c, c1
    CALL apoc.merge.relationship(c, toUpper(row.relationship_type), {}, {}, c1)
    YIELD rel
    RETURN count(*) AS count
  }
  RETURN count(*) AS finalCount
} IN TRANSACTIONS OF 10 ROWS
RETURN count(*)
""")

,count(*)
0,107


In [27]:
run_query("""
MATCH (c:Character|Harkonnen)
RETURN count(*)
""")

,count(*)
0,79


In [28]:
run_query("""
MATCH (c:Character&Harkonnen)
RETURN count(*)
""")

,count(*)
0,15


In [29]:
run_query("""
MATCH (c:Character&!Harkonnen)
RETURN count(*)
""")

,count(*)
0,64


In [40]:
run_query("""
MATCH (c:Character WHERE c.Culture = "Fremen" AND c.Died IS NOT NULL)
RETURN c.name AS character
""")

,character
0,Jamis
1,Liet Kynes
2,Mapes


In [45]:
run_query("""
MATCH p=(c:Character WHERE c.Culture = "Fremen" AND c.Died IS NOT NULL)-[r:ALLIES|FAMILY]-()
RETURN [n in nodes(p) | n.name] AS pairs, type(r) AS results
""")

,pairs,results
0,"[Jamis, Shimoon]",ALLIES
1,"[Jamis, Nisai]",FAMILY
2,"[Jamis, Stilgar]",FAMILY
3,"[Liet Kynes, Pardot Kynes]",FAMILY
4,"[Liet Kynes, Chani Kynes]",FAMILY
5,"[Liet Kynes, Paul Atreides]",ALLIES
6,"[Liet Kynes, Leto Atreides I]",ALLIES
7,"[Mapes, Paul Atreides]",ALLIES


In [46]:
run_query("""
MATCH p=(c:Character WHERE c.Culture = "Fremen" AND c.Died IS NOT NULL)-[r:!ALLIES|FAMILY]-()
RETURN [n in nodes(p) | n.name] AS pairs, type(r) AS results
""")

,pairs,results
0,"[Jamis, Nisai]",FAMILY
1,"[Jamis, Stilgar]",FAMILY
2,"[Jamis, Geoff]",ENEMIES
3,"[Liet Kynes, Uliet]",ENEMIES
4,"[Liet Kynes, Pardot Kynes]",FAMILY
5,"[Liet Kynes, Chani Kynes]",FAMILY


In [47]:
run_query("""
MATCH p=(c:Character WHERE c.Culture = "Fremen" AND c.Died IS NOT NULL)-[r:!(ALLIES|FAMILY)]-()
RETURN [n in nodes(p) | n.name] AS pairs, type(r) AS results
""")

,pairs,results
0,"[Jamis, Geoff]",ENEMIES
1,"[Liet Kynes, Uliet]",ENEMIES


In [51]:
run_query("""
MATCH (c:Character)
WHERE exists { (c)-[:FAMILY]-() }
RETURN count(*) AS count
""")

,count
0,25


In [52]:
run_query("""
MATCH (c:Character)
WHERE exists { MATCH (c)-[:FAMILY]-(t)
               WHERE (t)-[:ALLIES]-() AND (t)-[:MARRIAGE]-() }
RETURN count(*) AS count
""")

,count
0,5


In [54]:
run_query("""
MATCH (c:Character)
WHERE count{ (c)--() } > 2
RETURN count(*) AS count
""")

,count
0,11
